# Seminar Quantitativ Microbiology 
# Metabolite Production with Strain Engineering

## Introduction


For a overview on Jupyter notebooks read [this review](https://www.nature.com/articles/d41586-018-07196-1). 

---

## Tutorial Steps
  * Set up of Python environment
    * Basic libraries(sys, pandas, numpy, matplotlib, zipfile, cobrapy)
  * Analysis of Genome Scale Metabolic Model
    * Retrieval of GSMM for *P. pastoris*
    * Flux variability of exchange reactions
    * Minimal medium composition
  * Experimental growth rate reproduction
    * Familiarizing with biomass composition reactions
    * Defining functions for correct biomass equation switch
    * Data retrieval
    * Simulation loop
    * Graphical output


## Set-up compute environment

Before we can analyse GSMM, we have adjust the python environment that it integrates the cobrapy toolbox and downloading the GSMM.

### Basic Python libraries 
Some libraries that facilitate data manipulation

In [ ]:
import sys # loading commands to control/navigate within the system architecture
# Loading pandas, a library for data manipulation
from os.path import join
import xlrd
import pandas as pd
# import lxml

# Loading numpy, a library fo manipulation of numbers
import numpy as np

# loading matplotlib, a library for visualization
import matplotlib.pyplot as plt
%matplotlib inline

# loading cobrapy, a library dedicated to the analysis of genome scale metabolic models
from cobra.io import read_sbml_model, write_sbml_model, load_matlab_model

# loading escher for metabolic network visualization
# import escher
# from escher import Builder
# from time import sleep
# escher.rc['never_ask_before_quit'] = True
# list of available maps
# print(escher.list_available_maps())

# loading Memote, quality assessment of GSMM
from memote import test_model, snapshot_report

print('System ready')

In [ ]:
from BioLabSim.ModuleVirtualOrganism.HostSetup import Host, Strain

myHost = Host('Pput', Metabolism=True)
myHost.show_BiotechSetting()

In [ ]:
myst = Strain('Mut1', myHost, RefStrain='WT')

In [ ]:
from BioLabSim.ModuleVirtualOrganism.Metabolism import Help_StrainCharacterizer
RctNewDF = Help_StrainCharacterizer(myHost.var_Host, myHost.WT.info_GenesDF, myHost.WT.info_Genome, myst.info_Genome, myHost.WT.var_Model)
RctNew = RctNewDF[RctNewDF['RctFlag']==True].index.values
print(RctNew)
# For reactions with reduced expression and positive flux: reduce the upper limit,
# For reactions with increased expression and positive flux: increase the lower limit
# for reactions with negative flux the limits are exchanged.
FluxPos = RctNew[tuple([RctNewDF.loc[RctNew, 'RefFlux']>0])]
FluxNeg = RctNew[tuple([RctNewDF.loc[RctNew, 'RefFlux']<0])]
# Finding increased and decreased fluxes
FluxInc = RctNew[RctNewDF.loc[RctNew, 'NewExpr'].values / RctNewDF.loc[RctNew, 'RefExpr'].values>1]
FluxDec = RctNew[RctNewDF.loc[RctNew, 'NewExpr'].values / RctNewDF.loc[RctNew, 'RefExpr'].values<1]
print('Positive: {}, Negative: {}'.format(FluxPos,FluxNeg))
print('Increase: {}, Decrease: {}'.format(FluxInc,FluxDec))

In [ ]:
myModel = myHost.WT.var_Model
# Comb.1: positive flux with increased expression -> increasing lower bound
PosIncInd = np.intersect1d(FluxPos,FluxInc)
for Indx in PosIncInd:
    myModel.reactions[Indx].lower_bound = RctNewDF.loc[Indx, 'NewExpr'] * RctNewDF.loc[Indx, 'Expr2Flux']
# Comb.2: positive flux with decreased expression -> decreasing upper bound
PosDecInd = np.intersect1d(FluxPos,FluxDec)
for Indx in PosDecInd:
    myModel.reactions[Indx].lower_bound = RctNewDF.loc[Indx, 'NewExpr'] * RctNewDF.loc[Indx, 'Expr2Flux']
# Comb.1: negaitve flux with increased expression -> decreasing lower bound
NegIncInd = np.intersect1d(FluxNeg,FluxInc)
for Indx in NegIncInd:
    myModel.reactions[Indx].lower_bound = RctNewDF.loc[Indx, 'NewExpr'] * RctNewDF.loc[Indx, 'Expr2Flux']
# Comb.1: positive flux with increased expression -> increasing lower bound
NegDecInd = np.intersect1d(FluxNeg,FluxDec)
for Indx in NegDecInd:
    myModel.reactions[Indx].lower_bound = RctNewDF.loc[Indx, 'NewExpr'] * RctNewDF.loc[Indx, 'Expr2Flux']


In [ ]:
myst.info_GeneDF

In [ ]:
from BioLabSim.ModuleMeasureOrganism.Sequencing import make_BaseCompare
myDiff = make_BaseCompare(myHost.WT.info_Genome, myst.info_Genome)
print(myDiff)

In [ ]:
myHost.WT.info_GenesDF['Fluxes'].iloc[23]

In [ ]:
import random
GenesDF = myHost.WT.info_GenesDF
FluxActive = GenesDF[GenesDF['Fluxes']!=0].index.values
MutateEnzyme = random.sample(list(FluxActive),3)
# print('active flux: {} selected for mutation: {}'.format(FluxActive,MutateEnzyme))
print(GenesDF['Promoter'].iloc[MutateEnzyme[0]])
B_i, B_s = -12, -7
RefTar = GenesDF['Promoter'].iloc[MutateEnzyme[0]][B_i:B_s]
MutTar = list(RefTar)
B_i, B_s = -12, -7
NumberMutations=2
Mutate_Pos = random.sample(range(len(RefTar)), NumberMutations)
for NuclPos in Mutate_Pos:
#     print(type(NuclPos))
    MutTar[NuclPos] = random.sample([Base for Base in Bases if Base is not RefTar[NuclPos]], 1)[0]
MutTar = ''.join(MutTar)
print(RefTar)
print(MutTar)

In [ ]:
NumberMutations=3
Mutate_Pos = random.sample(range(5), NumberMutations)
Base_Exist = [Base for Base in [Extract[Pos] for Pos in Mutate_Pos]]
MutBas = [Base_Pos for Base_Pos in Base_Exist]

print(Mutate_Pos)
print(Base_Exist)
MutBas

In [ ]:
myStrainID = 'test'
myStrainGenome = myHost.info_Genome
Rct_Idx = myHost._Mutant__GenesDF[myHost._Mutant__GenesDF['RctID']=='PFK'].index.values
Rct_ORF = ''.join(str(n) for n in myHost._Mutant__GenesDF['ORF'].iloc[Rct_Idx].to_list())
Rct_RefProm = myHost._Mutant__GenesDF['Promoter'].iloc[Rct_Idx].values

# Extraction of reaction association promoter sequence from input genome
Gene_ORF = myHost.info_Genome.find(Rct_ORF)
target = myHost.info_Genome[Gene_ORF-14:Gene_ORF-9:1]
myStrainGenome = myStrainGenome.replace(target,'GGGGG',1)
myStrainGenome[Gene_ORF-14:Gene_ORF-9:1]
strainDict = {'StrainID': myStrainID, 'Genome':myStrainGenome}
myHost.info_Strain[myStrainID] = {}
myHost.info_Strain[myStrainID]['Genome'] = myStrainGenome
myHost.info_Strain['Reference'] = {}
myHost.info_Strain['Reference']['Genome'] = myHost.info_Genome
myHost.show_BiotechSetting()

In [ ]:
from BioLabSimFun import make_UpdateExpression, Help_StrainCharacterizer
RctNew_df = Help_StrainCharacterizer(myHost, 'test') # make_UpdateExpression(myHost, 'test', 'PFK')


In [ ]:
class person:
    def __init__(self,name):
        self.name = name
   

In [ ]:
p = person()
p.name


In [ ]:
p.db.display()

In [ ]:
Rct_ChangeIdx = RctNew_df[RctNew_df['RctFlag']==True].index.values
print('Reference expression: {}'.format(myHost._Mutant__GenesDF['Expression'].iloc[Rct_ChangeIdx].values))
print('Mutant expression: {}'.format(RctNew_df['Activity'].iloc[Rct_ChangeIdx].values))


In [ ]:
mycorr = myHost._Mutant__GenesDF['Fluxes'].values/myHost._Mutant__GenesDF['Expression'].values
mycorr

In [ ]:
from cobra.util.array import create_stoichiometric_matrix
StoichMat = create_stoichiometric_matrix(myHost.var_Model, array_type="dense", dtype=None)
metlist = [metabolite.id for metabolite in myHost.var_Model.metabolites]
solution = myHost.var_Model.optimize()

mymult = np.where(StoichMat*solution.fluxes.values!=0,1,0)
metInd = np.nonzero(np.sum(mymult, axis=1))
# metInd = np.reshape(np.array(np.nonzero(np.sum(mymult,axis=0))),-1)
np.array(metlist)[metInd]

In [ ]:
!wget http://bigg.ucsd.edu/static/models/e_coli_core.xml

# loading a visualization file of the metabolic network. 
# For frequently used models, like iJO1366, Escher automatically retrieves the visualization file.  
#!wget http://bigg.ucsd.edu/static/models/iJO1366.json

# generating cobra variable from SBML/xml/mat file
myFile = 'e_coli_core.xml'
model = read_sbml_model(myFile)

# solution = model.optimize()
model.summary() # fva=.95 additional argument specifies allowed deviation from the optimum 


### Malate Export Reaction
coding for malate export reaction.

In [ ]:
import random

Model = myhost.var_Model
# coding sequence construction
# first we determine the minimum coding gene length of nucleotides to distinguish the enzymes in the model
Enzyme_Number = len(Model.reactions)
Gen_Minimum = np.ceil(np.log2(Enzyme_Number))

# we want to represent codon triplicates, we calculate the next highest divisor of three
Gene_Length = int(np.ceil(Gen_Minimum/3))
CodonFile = 'CodonTriplets.csv'
CodonTriplets = pd.read_csv(CodonFile, delimiter=';', skipinitialspace=True)
CodonStop = CodonTriplets[['Stop' in s for s in CodonTriplets['Name']]].reset_index()
CodonCoding = CodonTriplets.drop(CodonStop.index).reset_index()
Gene_Coding = [random.choices(CodonCoding['Triplet'], weights=CodonCoding['Percent']) for CodonId in range(Gene_Length)]
Gene_Stop = random.choices(CodonStop['Triplet'], weights=CodonStop['Percent'])
Gene_Coding.append(Gene_Stop)
Gene_Coding.insert(0,['ATG'])
Gene_Coding = ''.join([Letter for Nest in Gene_Coding for Letter in Nest])
print(Gene_Coding)


In [ ]:
import pickle
import os
import random
# generating promoter sequence

# loading information that determines the exploratory space of the regressor
# weight file has boolean representations for each base on each position whether it was part of the training set or not
WeightFile = os.path.join('Models','NucleotideWeightTable.pkl')
with open(WeightFile, 'rb') as handle:
    Nucleotides_Weight = pickle.load(handle)

# the reference sequence contains the most common tested nucleotides at each position
RefFile = os.path.join('Models','RefSeq.txt')
with open(RefFile) as f:
    RefSeq = f.read()
Nucleotides_Weight.index = Nucleotides_Weight.index+40

# setup of default sequence with reference
myAr = np.asarray([Letter for Letter in RefSeq])

# changing positions within the exploratory space of the regressor
Bases = ['A','C','G','T']
for idx in range(len(Nucleotides_Weight)):
    np.put(myAr,Nucleotides_Weight.index[idx],random.choices(Bases,Nucleotides_Weight.iloc[idx].values))
Gene_Promoter = ''.join(myAr)
print('Promoter: {}'.format(Gene_Promoter))

In [ ]:
# test of expression strength

import joblib
from BioLabSimFun import list_onehot, list_integer

print(len(Gene_Promoter))

Data_Folder = 'ExpressionPredictor'
if myhost.var_Host == 'Ecol':
    Regressor_File = os.path.join(Data_Folder,'Ecol-Promoter-predictor.pkl')
    Add_Params = os.path.join(Data_Folder,'Ecol-Promoter-AddParams.pkl')
    Scaler_DictName = 'Ecol Promoter Activity_Scaler'
elif myhost.var_Host == 'Pput':
    Regressor_File = os.path.join(Data_Folder,'Ptai-Promoter-predictor.pkl')
    Add_Params = os.path.join(Data_Folder,'Ptai-Promoter-AddParams.pkl')
    Scaler_DictName = 'Ptai Promoter Activity_Scaler'
else:
    print('Non-recognized host name. Rename host to either "Ecol" or "Pput."')

Predictor = joblib.load(Regressor_File)
Params = pickle.load(open(Add_Params, 'rb'))
Positions_removed = Params['Positions_removed']
# Expr_Scaler = Params[Scaler_DictName]

X = np.array(list_onehot(np.delete(list_integer(Gene_Promoter),Positions_removed, axis=0))).reshape(1,-1)  
# GC content calculation
GC_cont = (Gene_Promoter.count('G') + Gene_Promoter.count('C'))/len(Gene_Promoter)
X = np.append(X,GC_cont) 
Y = Predictor.predict(np.array([X]))
Y
# Expression = round(float(Expr_Scaler.inverse_transform(Y_Test_norm)),3)


In [ ]:
# Load an Escher map
builder.map_name = 'e_coli_core.Core metabolism'
builder.model = model
builder.reaction_data = solution.fluxes
# Add some data for metabolites
#builder.metabolite_data = solution.shadow_prices
# Simplify the map by hiding some labels
builder.hide_secondary_metabolites = True
#builder.hide_all_labels = True
builder.reaction_scale = [
    { 'type': 'min', 'color': '#000000', 'size': 12 },
    { 'type': 'median', 'color': '#ffffff', 'size': 20 },
    { 'type': 'max', 'color': '#ff0000', 'size': 25 }
]
builder.reaction_scale_preset = 'GaBuRd'

# Make all the arrows three times as thick
builder.reaction_scale = [
    {k: v * 3 if k == 'size' else v for k, v in x.items()}
    for x in builder.reaction_scale
]
builder.save_html('escher_map_file.html')

## Prediction of growth rates
### Choose the Substrate

In [ ]:
myhost.Choose_Substrate(None)
myhost.show_BiotechSetting()

### Familiarizing with biomass composition reactions

Microorganisms adapt to their substrate. Different substrates provide different energy content and require different cellular resources to become metabolized. In GSMM these differences may be represented by different equations/reactions for the substrates. In iMR1026 for *P. pastoris* there are various biomass equations for glucose, glycerol, glucose-glycerol mixtures, and methanol. When simulating a model, we have to make sure we use the right biomass equation fitting with the substrate.

In [ ]:
# List of all reactions with 'BIOMASS' in their name
model.reactions.query('?..?')

In [ ]:
# Looking in detail to biomass with methanol; use the reaction name given to you from the list of all biomass reactions;
model.reactions.?..?

### Defining functions for correct biomass equation switch

For each substrate, the boundary exchange fluxes are activated and the reactions of competing substrates are disabled.

Remember to write a specific function to adapt the model for glucose utilization.

In [ ]:
def AdaptMethanol(model, meoh_up):
  model.objective = 'BIOMASS_meoh'
  # setting uptake reactions right
  model.reactions.Ex_glc_D.lower_bound = 0
  model.reactions.Ex_glyc.lower_bound = 0
  model.reactions.ATPM.lower_bound = .4
  model.reactions.Ex_meoh.lower_bound = -np.abs(meoh_up)
  # setting additional biomass composition
  model.reactions.LIPIDS_meoh.upper_bound = 1000
  model.reactions.PROTEINS_meoh.upper_bound = 1000
  model.reactions.STEROLS_meoh.upper_bound = 1000
  model.reactions.BIOMASS_meoh.upper_bound = 1000
  # deactivating Glyc-based biomass composition
  model.reactions.LIPIDS_glyc.upper_bound = 0
  model.reactions.PROTEINS_glyc.upper_bound = 0
  model.reactions.STEROLS_glyc.upper_bound = 0
  model.reactions.BIOMASS_glyc.upper_bound = 0  
  # deactivating Glc-based biomass composition
  model.reactions.LIPIDS.upper_bound = 0
  model.reactions.PROTEINS.upper_bound = 0
  model.reactions.STEROLS.upper_bound = 0
  model.reactions.BIOMASS.upper_bound = 0  
  return model

def AdaptGlycerol(model, glyc_up):
  model.objective = 'BIOMASS_glyc'
  # setting uptake reactions right
  model.reactions.Ex_meoh.lower_bound = 0
  model.reactions.Ex_glc_D.lower_bound = 0
  model.reactions.ATPM.lower_bound = 2.5
  model.reactions.Ex_glyc.lower_bound = -np.abs(glyc_up)
  # setting additional biomass composition
  model.reactions.LIPIDS_glyc.upper_bound = 1000
  model.reactions.PROTEINS_glyc.upper_bound = 1000
  model.reactions.STEROLS_glyc.upper_bound = 1000
  model.reactions.BIOMASS_glyc.upper_bound = 1000  
  # deactivating MeOH-based biomass composition
  model.reactions.LIPIDS_meoh.upper_bound = 0
  model.reactions.PROTEINS_meoh.upper_bound = 0
  model.reactions.STEROLS_meoh.upper_bound = 0
  model.reactions.BIOMASS_meoh.upper_bound = 0
  # deactivating Glc-based biomass composition
  model.reactions.LIPIDS.upper_bound = 0
  model.reactions.PROTEINS.upper_bound = 0
  model.reactions.STEROLS.upper_bound = 0
  model.reactions.BIOMASS.upper_bound = 0
  return model

def AdaptGlucose(model, glc_up):
  # setting uptake reactions right
  model.reactions.Ex_meoh.lower_bound = 0
  model.reactions.Ex_glyc.lower_bound = 0
  model.reactions.Ex_glc_D.lower_bound = -np.abs(glc_up)
  # setting additional biomass composition
  model.reactions.LIPIDS.upper_bound = 1000
  model.reactions.PROTEINS.upper_bound = 1000
  model.reactions.STEROLS.upper_bound = 1000
  model.reactions.BIOMASS.upper_bound = 1000  
  # deactivating Glyc-based biomass composition
  model.reactions.LIPIDS_glyc.upper_bound = 0
  model.reactions.PROTEINS_glyc.upper_bound = 0
  model.reactions.STEROLS_glyc.upper_bound = 0
  model.reactions.BIOMASS_glyc.upper_bound = 0  
 # deactivating meoh-based biomass composition
  model.reactions.LIPIDS_meoh.upper_bound = 0
  model.reactions.PROTEINS_meoh.upper_bound = 0
  model.reactions.STEROLS_meoh.upper_bound = 0
  model.reactions.BIOMASS_meoh.upper_bound = 0
  model.objective = 'BIOMASS'
  return model

### Data retrieval

For evaluation of the growth rate prediction of the *P. pastoris* model we use experimental data from the closely related organism *Ogataea polymorpha*. The measurements in the table are extracted from [van Dijken et al. 1976](https://dx.doi.org/10.1007/bf00446560) for methanol and from [de Koning et al., 1987](https://dx.doi.org/10.1007/BF00456710) and [Moon et al., 2003](https://dx.doi.org/10.1385/ABAB:111:2:65).

Data Address: [here](https://rwth-aachen.sciebo.de/s/o72jwWQWh3ame1e/download)

**When using Binder:**
Click on "here" and save the file. Go back to "Home"-Tab in the browser afterwards and navigate back to the root directory of your cloud system (i.e. you see the Jupyter notebook files). Click on the "Upload" button on the right side of the window and select the file you just saved. Now you should see the file "Opol-expt-grwth_MeOH-Glyc.csv", click again on "Upload" in the line of the corresponding file. Now go back to the simulation-Tab. Replace '?..?' in the cell below by entering the name of the csv file you downloaded. Then press Ctrl+Enter to load the experimental data.

**When using Colab:**
Click on "here" and save the file. Then click on "Files" on the left side of the window. Upload the file you have just saved by clicking on "Upload" and selecting this file. Now you should see the file "Opol-expt-grwth_MeOH-Glyc.csv" in the "Files" section. If this is not the case, click on "Refresh". Then right-click on this file and select "Copy path". Replace '?..?' in the cell below with the file by pressing Ctrl+V. Then evaluate the cell by Ctrl+Enter.

In [ ]:
# Create Excel file with all uptake rates and then import it instead as follows:
# data = pd.read_excel('nameOfFile.xlsx', sheet_name='Ecol', index_col=0)

data = pd.read_csv('?..?', delimiter=',|;', engine='python')
data


Adding measurements for glucose from [Lehnen et al., 2017](https://doi.org/10.1016/j.meteno.2017.07.001) to our basic data table. The required information is the growth rate on glucose for *H. polymorpha*. Extract the necessary information from Table 3 in the article. The reaction name for the 'Exchange'-reaction is 'Ex_glc_D', replace this in the corresponding position of '?..?' in the cell below.

In [ ]:
data = data.append({'Substrate':'Glucose', 'Exchange':'?..?', 'uptake rate (mmol/gCDW/h)':?..?, 'growth rate (/h)':?..?, 'source':'Lehnen et al.'}, ignore_index=True)

### Simulation loop
For-Loop over all experimental data points.

Remember to add a decision when you include glucose.

In [ ]:
# At the beginning a substrate was selected, therefore only one execution is necessary, approximately as follows:
# Substrate = myhost.var_Substrate
# model=AdaptSubstrate(model, data.loc[['Substrate'],['uptake rate']].values)
# growth_simulated = model.optimize()

growth_simulated = [];
# test_model = model.copy()
# iteration over all rows in 'data'
for index, row in data.iterrows():
  with model as test_model:
    print(index) # printing the row number to get feedback that everything is working
    # selecting the right substrate in the model based on 'Substrate' in 'data'
    if row['Substrate'] == 'Methanol':
      model = AdaptMethanol(test_model, row['uptake rate (mmol/gCDW/h)'])
    elif row['Substrate'] == 'Glycerol':
      model = AdaptGlycerol(test_model, row['uptake rate (mmol/gCDW/h)'])
    elif row['Substrate'] == 'Glucose':
      test_model = AdaptGlucose(test_model, row['uptake rate (mmol/gCDW/h)'])
    else:
      print('substrate not considered')      
  #     model.optimize()
    growth_simulated.append(model.slim_optimize())


### Graphical output

Remember to add glucose to the visualization. Add the right axis labels, and a file name.

In [ ]:
plt.scatter(data['growth rate (/h)'][0:7], growth_simulated[0:7], s=50, c='k', marker='o');
plt.scatter(data['growth rate (/h)'][7], growth_simulated[7], s=50, c='k', marker='s');
plt.scatter(data['growth rate (/h)'][8], growth_simulated[8], s=50, c='k', marker='d');
plt.xlabel('?..?');
plt.ylabel('?..?');
myline = np.linspace(0,np.max(growth_simulated),10);
plt.plot(myline,myline,'k--');
plt.title('Growth rate comparison');
plt.legend(['Optimum', 'Methanol (van Dijken)', 'Glycerol (deKoning, Moon)', 'Glucose (Lehnen et al.)'], loc=2);
plt.style.use('seaborn-paper')

# Saving figure
plt.savefig('?..?.png')